<h1> Regional malaria data with monthly weather information </h1>

In [1]:
from __future__ import division
import numpy as np
import json
import re
import gzip
import pandas as pd
%pylab inline
pylab.rcParams['figure.figsize'] = (12.0, 8.0) # set size of figures"
plt.rcParams.update({'font.size': 24})
import seaborn as sns
import datetime
import csv

from ethiopian_date import ethiopian_date

Populating the interactive namespace from numpy and matplotlib


In [46]:
def add_one_month(Date):
    if Date.month < 12:
        year = Date.year
        month = Date.month + 1
    else:
        year = Date.year + 1
        month = 1
    return pd.datetime(year, month, Date.day)

def sub_one_month(Date):
    if Date.month > 1:
        year = Date.year
        month = Date.month - 1
    else:
        year = Date.year - 1
        month = 12
    return pd.datetime(year, month, Date.day)

def set_to_first_of_month(Date):
    return datetime.date(Date.year, Date.month, 1)

def add_five_days(Date):
    '''Incase we have to get out of the 13th month Quaggimi to avoid an error'''
    # First. is less than 25 and not Feb, we can add 5 days. and change nothing else
    # If the 26th or more and not Feb... check if December, add a year set month to 1. Not Dec, add a month
    if (Date.day <= 25) and (Date.month != 2):
        year = Date.year
        month = Date.month
        day = Date.day + 6
    elif (Date.day > 25) and (Date.month != 2):
        if Date.month == 12:
            year = Date.year + 1
            month = 1
            day = max(1, 31 - Date.day)
        else:
            year = Date.year
            month = Date.month + 1
            day = max(1, 30 - Date.day)
    elif (Date.day > 23) and (Date.month == 2):
        year = Date.year
        month = Date.month + 1
        day = max(1, 28 - Date.day)
    else:
        year = Date.year
        month = Date.month
        day = Date.day + 6
    return pd.datetime(year, month, day)

<h1> Load in population data </h1>

In [62]:
DF_pop = pd.read_excel('/Users/attiladobi/Documents/Ethiopia_pop.xlsx', header=2)
DF_pop.set_index('Map', inplace=True)
DF_pop.index = [name.lower() for name in DF_pop.index]
#convert population from string to int 
DF_pop['Population'] = [int(pop.replace(',','')) for pop in DF_pop['Population']]

<h1> Load in weather data (global monthly average) </h1>

In [64]:
DF_weather = pd.read_excel('/Users/attiladobi/Documents/Ethiopia_ave_weather.xlsx', header=1).dropna(how='all').set_index(['city','Field'])

<h1> add average weather data for population df </h1>

In [65]:
DF_pop['ave_temp'] = [DF_weather['ANNUAL'][cap]['temp'] for cap in DF_pop['Capital']]
DF_pop['ave_prec_mm'] = [DF_weather['ANNUAL'][cap]['prec_mm'] for cap in DF_pop['Capital']]

<h1> load in Regional and Woreda level map </h1>

In [66]:
# Read in Lat Lon as strings to avoid double precision issues 
lat_lon_map = dict()
# RegionName, RegionLat, RegionLon
fpath = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/region_mapped.csv'
with open(fpath, 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    # skip one row
    spamreader.next()
    for row in spamreader:
        lat_lon_map[row[0].lower()] = (row[1], row[2])
lat_lon_map

{'addis ababa': ('8.980483001254077', '38.78553834507309'),
 'afar': ('12.036445063176075', '40.772735414805204'),
 'amhara': ('11.5649524790046', '38.04353615460529'),
 'benishangul-gumuz': ('10.502924415157686', '35.44030701966273'),
 'dire dawa': ('9.606269193503696', '42.003026894902156'),
 'gambella': ('7.683850454124325', '34.33675414861345'),
 'harari': ('9.289660023774802', '42.172525872858124'),
 'oromiya': ('7.323567398452454', '38.66052453790127'),
 'snnpr': ('6.465808724769859', '36.806469462905675'),
 'somali': ('6.932778405509416', '43.32988331430246'),
 'tigray': ('13.777114742142253', '38.43867332133363')}

<h1> load in indicator ID mapping </h1>

In [67]:
import sys
sys.path.append('/Users/attiladobi/zenysis')
import config.et.indicators

useful_groups = config.et.indicators.HMIS_GROUPS + \
    config.et.composite_indicators.HMIS_GROUPS + \
    config.et.indicators.NNPMT_GROUPS + \
    config.et.indicators.HSTP_GROUPS + \
    [config.et.indicators.CALCULATED_INDICATOR_GROUP] + config.et.indicators.SPA_GROUPS
IDlookup = {val['id']:val for cluster in useful_groups for val in cluster['indicators']}

<h1> Load in region level malaria data </h1>

In [68]:
# with open('/Users/attiladobi/Downloads/Malaria_region_20170203.json', 'rb') as f:
with open('/Users/attiladobi/Downloads/2017-03-07_malaria_region.json', 'rb') as f:
    data = f.readlines()
# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
DF2 = pd.read_json(data_json_str)

In [69]:
df_list = []
for region in DF2['byGeo'][0].keys():
    region_name = region.split('_')[0]
    meta_data = pd.DataFrame(DF2['byGeo'][0][region]['data']['month'])
    formatted_data = pd.DataFrame(meta_data[1:].stack()).reset_index(level=0)
    formatted_data.index = [pd.to_datetime(val) for val in formatted_data.index]
    formatted_data.columns = ['Indicator_id', 'val']
    formatted_data.index.name = 'Real_Date'
    formatted_data['Region'] = region_name
    # Drop data that doesn't have indicator information. They start with Real_Date
    formatted_data = formatted_data[formatted_data['Indicator_id'] != 'Real_Date']
    df_list.append(formatted_data)
DF_ind=pd.concat(df_list)
DF_ind['Indicator'] = [IDlookup[val]['text'] for val in DF_ind['Indicator_id']]

#Note: region_name, zone_name, woreda_name are all lower case

In [70]:
DF_ind.groupby('Indicator').count()

,Indicator_id,val,Region
Indicator,,,
Number of HHs that need LLINs in the last 12 months,11,11,11
Number of slides or RDT positive for malaria,484,484,484
Number of slides or RDT positive for malaria 5-14 years: Females,340,340,340
Number of slides or RDT positive for malaria 5-14 years: Males,340,340,340
Number of slides or RDT positive for malaria < 5 years: Females,341,341,341
Number of slides or RDT positive for malaria < 5 years: Males,341,341,341
Number of slides or RDT positive for malaria >=15 years: Females,339,339,339
Number of slides or RDT positive for malaria >=15 years: Males,340,340,340
Number of targeted HHs received at least one LLINs in the last 12 months,11,11,11


<h3> look up woreda lat and log based on region </h3>

In [71]:
DF_ind['RegionLat'] = [lat_lon_map[region][0] for region in DF_ind['Region']]
DF_ind['RegionLon'] = [lat_lon_map[region][1] for region in DF_ind['Region']]

<h1> Label gender and age catagories </h1>

In [72]:
# Indicators used for fitting
ind_total = 'Number of slides or RDT positive for malaria'
ind_f_5 = 'Number of slides or RDT positive for malaria < 5 years: Females'
ind_m_5 = 'Number of slides or RDT positive for malaria < 5 years: Males'
ind_f_5_14 = 'Number of slides or RDT positive for malaria 5-14 years: Females'
ind_m_5_14 = 'Number of slides or RDT positive for malaria 5-14 years: Males'
ind_f_15_up = 'Number of slides or RDT positive for malaria >=15 years: Females'
ind_m_15_up = 'Number of slides or RDT positive for malaria >=15 years: Males'

ind_names = [ind_total, ind_f_5, ind_m_5, ind_f_5_14, ind_m_5_14, ind_f_15_up, ind_m_15_up]
ind_age_cat = [4, 1, 1, 2, 2, 3, 3]
ind_gender_cat = [3, 2, 1, 2, 1, 2, 1]
name_to_gen_age_map = {ind_name: (gender, age) for ind_name, gender, age in zip(ind_names, ind_gender_cat, ind_age_cat)}

def lookup_gen_age(ind_name):
    if ind_name in name_to_gen_age_map:
        return name_to_gen_age_map[ind_name]
    else:
        return (-1, -1)
    
DF_ind['gen_cat'] = [lookup_gen_age(ind_name)[0] for ind_name in DF_ind['Indicator']]
DF_ind['age_cat'] = [lookup_gen_age(ind_name)[1] for ind_name in DF_ind['Indicator']]

<h1> add Ethiopian dates </h1>

In [73]:
DF_ind = DF_ind.replace(0,nan).dropna()
#DF_ind = DF_ind.dropna()
e_dates = []
for date in DF_ind.index:
    ''' Except error if we land in the 13th month. Shift the date by 5 days'''
    try:
        e_dates.append(ethiopian_date.EthiopianDateConverter.date_to_ethiopian(date))
    except ValueError:
        #print date, add_five_days(date)
        e_dates.append(ethiopian_date.EthiopianDateConverter.date_to_ethiopian(add_five_days(date)))
        
DF_ind['E_Date'] = [set_to_first_of_month(date) for date in e_dates] # another path for the 13th month
DF_ind['E_Date'] = DF_ind['E_Date'].astype(datetime64)
DF_ind['E_year'] = [val.year for val in DF_ind['E_Date']]
DF_ind['E_month'] = [val.month for val in DF_ind['E_Date']]
DF_ind['month'] = DF_ind.index.month
DF_ind['year'] = DF_ind.index.year

<h1> Add population and density data </h1>

In [74]:
DF_ind['capital'] = [DF_pop.loc[key]['Capital'] for key in DF_ind['Region']]
DF_ind['pop'] = [DF_pop['Population'][key] for key in DF_ind['Region']]
DF_ind['density'] = [DF_pop['Density'][key] for key in DF_ind['Region']]
DF_ind['ave_temp'] = [DF_pop['ave_temp'][key] for key in DF_ind['Region']]
DF_ind['ave_prec_mm'] = [DF_pop['ave_prec_mm'][key] for key in DF_ind['Region']]
DF_ind['elevation_m'] = [DF_pop['Elevation_m'][key] for key in DF_ind['Region']]
DF_ind['infection_rate'] = DF_ind['val'] / DF_ind['pop'] #only applies to one indicator: indicator = 'Number of slides or RDT positive for malaria'

#is capital?
DF_ind['is_capital'] = [0 if region != 'addis ababa' else 1 for region in DF_ind['Region']]

<h1> Add insecticide net fraction data </h1> 

In [75]:
indicator = 'Number of slides or RDT positive for malaria'
DF_mean = DF_ind.loc[DF_ind['Indicator']==indicator].groupby('Region').mean()

ind_nets = 'Proportion of targeted HH covered with LLIN in the last 12 months'
DF_mean_nets = DF_ind.loc[DF_ind['Indicator']==ind_nets].groupby('Region').mean()
#add back Addis info:
DF_mean_nets.loc['addis ababa'] =  DF_mean_nets.loc['oromiya']
# this is the only one used
DF_ind['net_frac'] = [DF_mean_nets.val[region] for region in DF_ind['Region']]

ind_nets_del = 'Number of targeted HHs received at least one LLINs in the last 12 months'
DF_net_del = DF_ind.loc[DF_ind['Indicator']==ind_nets_del].groupby('Region').mean()

ind_nets_needed = 'Number of HHs that need LLINs in the last 12 months'
DF_net_needed = DF_ind.loc[DF_ind['Indicator']==ind_nets_needed].groupby('Region').mean()

DF_net_diff = (DF_net_del - DF_net_needed) / DF_net_needed

<h1> Add average monthly weather data </h1>

In [76]:
month_map = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12 }
inv_month_map = {v: k for k, v in month_map.iteritems()}

DF_ind['ave_temp'] = [DF_weather.loc[capital,'temp'][inv_month_map[month]] for capital,month in zip(DF_ind['capital'],DF_ind['month'])]
DF_ind['ave_prec_mm'] = [DF_weather.loc[capital,'prec_mm'][inv_month_map[month]] for capital,month in zip(DF_ind['capital'],DF_ind['month'])]
DF_ind['ave_prec_mm'] = DF_ind['ave_prec_mm'].replace(nan,0)

In [77]:
DF_ind

,Indicator_id,val,Region,Indicator,RegionLat,RegionLon,gen_cat,age_cat,E_Date,E_year,...,year,capital,pop,density,ave_temp,ave_prec_mm,elevation_m,infection_rate,is_capital,net_frac
Real_Date,,,,,,,,,,,,,,,,,,,,,
2014-07-08,composite_indicator_47,5786.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2006-11-01,2006,...,2014,Semera,1723000,19.58,31.9,46.4,614,0.003358,0,0.615103
2014-08-07,composite_indicator_47,5760.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2006-12-01,2006,...,2014,Semera,1723000,19.58,31.2,55.4,614,0.003343,0,0.615103
2014-09-06,composite_indicator_47,7952.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-01-01,2007,...,2014,Semera,1723000,19.58,30.0,14.2,614,0.004615,0,0.615103
2014-10-11,composite_indicator_47,9759.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-02-01,2007,...,2014,Semera,1723000,19.58,28.2,4.4,614,0.005664,0,0.615103
2014-11-10,composite_indicator_47,10696.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-03-01,2007,...,2014,Semera,1723000,19.58,26.2,5.9,614,0.006208,0,0.615103
2014-12-10,composite_indicator_47,11724.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-04-01,2007,...,2014,Semera,1723000,19.58,24.7,3.1,614,0.006804,0,0.615103
2015-01-09,composite_indicator_47,11493.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-05-01,2007,...,2015,Semera,1723000,19.58,24.7,7.4,614,0.006670,0,0.615103
2015-02-08,composite_indicator_47,9593.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-06-01,2007,...,2015,Semera,1723000,19.58,25.6,7.3,614,0.005568,0,0.615103
2015-03-10,composite_indicator_47,9635.000000,afar,Number of slides or RDT positive for malaria,12.036445063176075,40.772735414805204,3,4,2007-07-01,2007,...,2015,Semera,1723000,19.58,27.3,28.9,614,0.005592,0,0.615103


<h1> save the data as csv </h1>

In [79]:
DF_ind.to_csv('Malaria_regional_3-15-2017.csv')

<h1> Load in json data with weather station LAT LON </h1>

In [30]:
def load_dict_from_json(filepath):
    '''given file path to json mapping return a dict'''
    with open(filepath) as data_file:    
        return json.load(data_file)
    
ET_STN = load_dict_from_json('ET_STATION_LIST.json')

In [197]:
ET_STN['634770']

{u'BEGIN': u'19570119',
 u'CODE': u'',
 u'ELEVATION': u'',
 u'END': u'19631207',
 u'LAT': u'+00.000',
 u'LON': u'+000.000',
 u'NAME': u'...',
 u'STN': u'634770',
 u'WBAN': u'99999'}

In [31]:
# Remove empty index of 000.000 coordinates
STATION_LAT_LON = [(key, float(val['LAT']), float(val['LON']) ) for key,val in ET_STN.iteritems() if ((val['LAT'] != '') and val['LAT'] != u'+00.000') ]

In [32]:
STATION_LAT_LON

[(u'628700', 8.129, 34.563),
 (u'634510', 8.733, 39.0),
 (u'634730', 9.333, 42.717),
 (u'633300', 13.467, 39.533),
 (u'634531', 8.933, 40.083),
 (u'634530', 8.867, 39.9),
 (u'634740', 7.133, 40.0),
 (u'634020', 7.666, 36.817),
 (u'634043', 10.019, 34.586),
 (u'634760', 7.15, 40.7),
 (u'630130', 15.55, 37.883),
 (u'634780', 5.935, 43.579),
 (u'633299', 14.147, 38.773),
 (u'634600', 7.067, 38.05),
 (u'634520', 9.417, 41.0),
 (u'635320', 5.867, 38.95),
 (u'634500', 8.978, 38.799),
 (u'633330', 11.083, 39.717),
 (u'634723', 6.367, 39.833),
 (u'633310', 12.52, 37.434),
 (u'635330', 5.283, 39.75),
 (u'634750', 6.75, 44.267),
 (u'634030', 8.167, 35.55),
 (u'633340', 10.35, 37.717),
 (u'633400', 9.083, 36.45),
 (u'634710', 9.625, 41.854),
 (u'635000', 6.039, 37.59),
 (u'633320', 11.608, 37.322)]

<h1> Find the nearest weather station </h1>

In [33]:
from geopy.distance import great_circle

def nearest_station(lat, lon):
    min_arc_distance_km = 10000;
    best_stn = ''
    # Loop though all possible stations
    for stn, stn_lat, stn_lon in STATION_LAT_LON:
        arc_distance_km = great_circle((lat, lon), (stn_lat, stn_lon)).km
        if arc_distance_km < min_arc_distance_km:
            min_arc_distance_km = arc_distance_km
            best_stn = stn
    return best_stn, min_arc_distance_km

In [34]:
Nearest = [nearest_station(float(Lat), float(Lon)) for Lat,Lon in DF_ind[['RegionLat', 'RegionLon']].values]
DF_ind['STN'] = [val[0] for val in Nearest]
DF_ind['STN_dist_km'] = [val[1] for val in Nearest]

<h1> Load in the NOAA dataframe </h1>

In [35]:
DF_ind['YEAR_MONTH'] = DF_ind.index.strftime('%Y-%m')

In [36]:
NOAA = pd.read_csv('/Users/attiladobi/Documents/NOAA_ET_LATLON.csv', index_col=0)
# Convert to String format, otherwise it is loaded as an int
NOAA['STN'] = NOAA['STN'].astype('unicode')

In [37]:
NOAA_mean = NOAA.groupby(['STN', 'YEAR_MONTH']).mean().reset_index()
NOAA_max = NOAA.groupby(['STN', 'YEAR_MONTH']).max().reset_index()
NOAA_min = NOAA.groupby(['STN', 'YEAR_MONTH']).min().reset_index()

In [38]:
DF_monthly_weather = pd.merge(DF_ind, NOAA_mean, how='left', on=['STN', 'YEAR_MONTH'] )

In [39]:
DF_monthly_weather.to_csv('Malaria_regional_monthly_weather.csv')

In [40]:
DF_monthly_weather.groupby(['Region', 'STN_dist_km']).count()

,,Indicator_id,val,Indicator,RegionLat,RegionLon,gen_cat,age_cat,E_Date,E_year,E_month,...,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT,ELEVATION,LAT,LON
Region,STN_dist_km,,,,,,,,,,,,,,,,,,,,,
addis ababa,1.504505,267,267,267,267,267,267,267,267,267,267,...,267,0,267,267,267,0,267,267,267,267
afar,156.463715,289,289,289,289,289,289,289,289,289,289,...,289,0,289,289,289,0,289,289,289,289
amhara,78.764050,295,295,295,295,295,295,295,295,295,295,...,295,0,295,295,295,0,295,295,295,295
benishangul-gumuz,107.887161,289,289,289,289,289,289,289,289,289,289,...,0,0,0,0,0,0,0,0,0,0
dire dawa,16.475079,269,269,269,269,269,269,269,269,269,269,...,269,0,269,269,269,0,269,269,269,269
gambella,55.432257,287,287,287,287,287,287,287,287,287,287,...,0,0,0,0,0,0,0,0,0,0
harreri,51.112314,289,289,289,289,289,289,289,289,289,289,...,289,0,289,289,289,0,289,289,289,289
oromiya,73.166150,292,292,292,292,292,292,292,292,292,292,...,263,0,292,292,292,0,292,292,292,292
snnpr,98.785010,292,292,292,292,292,292,292,292,292,292,...,242,0,292,292,292,0,292,292,292,292


In [311]:
DF_monthly_weather.groupby('STN').count()

,Indicator_id,val,Region,Indicator,RegionLat,RegionLon,gen_cat,age_cat,E_Date,E_year,...,MXSPD,GUST,MAX,MIN,PRCP,SNDP,FRSHTT,ELEVATION,LAT,LON
STN,,,,,,,,,,,,,,,,,,,,,
628700,233,233,233,233,233,233,233,233,233,233,...,0,0,0,0,0,0,0,0,0,0
633299,233,233,233,233,233,233,233,233,233,233,...,0,0,0,0,0,0,0,0,0,0
633320,233,233,233,233,233,233,233,233,233,233,...,233,0,233,233,233,0,233,233,233,233
633330,233,233,233,233,233,233,233,233,233,233,...,233,0,233,233,233,0,233,233,233,233
634043,233,233,233,233,233,233,233,233,233,233,...,0,0,0,0,0,0,0,0,0,0
634500,231,231,231,231,231,231,231,231,231,231,...,231,0,231,231,231,0,231,231,231,231
634600,233,233,233,233,233,233,233,233,233,233,...,209,0,233,233,233,0,233,233,233,233
634710,449,449,449,449,449,449,449,449,449,449,...,449,0,449,449,449,0,449,449,449,449
634750,193,193,193,193,193,193,193,193,193,193,...,0,0,0,0,0,0,0,0,0,0


In [42]:
DF_monthly_weather.groupby('Indicator').count().index

Index([u'Email or internet access',
       u'Facility connected to the central power supply/grid',
       u'Land line telephone', u'Malaria positivity rate',
       u'Number of HHs that need LLINs in the last 12 months',
       u'Number of Households utilizing latrine',
       u'Number of slides or RDT positive for malaria',
       u'Number of slides or RDT positive for malaria 5-14 years: Females',
       u'Number of slides or RDT positive for malaria 5-14 years: Males',
       u'Number of slides or RDT positive for malaria < 5 years: Females',
       u'Number of slides or RDT positive for malaria < 5 years: Males',
       u'Number of slides or RDT positive for malaria >=15 years: Females',
       u'Number of slides or RDT positive for malaria >=15 years: Males',
       u'Number of targeted HHs received at least one LLINs in the last 12 months',
       u'Proportion of households' access to latrine facilities',
       u'Proportion of targeted HH covered with LLIN in the last 12 months'